In [1]:
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.RegressionEvaluator

In [2]:
val rawData = sc.textFile("/home/jovyan/data/bike-sharing/hour.csv")
val header = rawData.first()
val records = rawData.filter(line => line != header)

rawData = /home/jovyan/data/bike-sharing/hour.csv MapPartitionsRDD[1] at textFile at <console>:30
header = instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
records = MapPartitionsRDD[2] at filter at <console>:32


MapPartitionsRDD[2] at filter at <console>:32

In [3]:
records.first()

1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16

In [4]:
// records.cache()

Name: Syntax Error.
Message: 
StackTrace: 

The other 3 are casual and registered and we are not going to bother about them.

In [5]:
case class BikeRentals(season: Int, yr: Int, mnth: Int, 
                       hr: Int, holiday: Int, weekday: Int,
                       workingday: Int, weathersit: Int, 
                       temp: Double, atemp: Double, hum: Double,
                       windspeed: Double, cnt: Double)
def parseString(recs: String): BikeRentals = {
    val fields = recs.split(",")
    assert(fields.size==17)
    BikeRentals(fields(2).toInt, 
                fields(3).toInt,
                fields(4).toInt,
                fields(5).toInt,
                fields(6).toInt,
                fields(7).toInt,
                fields(8).toInt,
                fields(9).toInt,
                fields(10).toDouble,
                fields(11).toDouble,
                fields(12).toDouble,
                fields(13).toDouble,
                fields(16).toDouble)
}

defined class BikeRentals


parseString: (recs: String)BikeRentals


In [6]:
val data = records.map(parseString).toDF()
data.show(3)

+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+----+
|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed| cnt|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+----+
|     1|  0|   1|  0|      0|      6|         0|         1|0.24|0.2879|0.81|      0.0|16.0|
|     1|  0|   1|  1|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0|40.0|
|     1|  0|   1|  2|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0|32.0|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+----+
only showing top 3 rows



data = [season: int, yr: int ... 11 more fields]


[season: int, yr: int ... 11 more fields]

In [7]:
val featureCols = Array("season", "yr", "mnth", "hr", "holiday", "weekday", "workingday", "weathersit", "temp", "atemp", "hum", "windspeed", "cnt")
val assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol("features")
val dataDF = assembler.transform(data)

featureCols = Array(season, yr, mnth, hr, holiday, weekday, workingday, weathersit, temp, atemp, hum, windspeed, cnt)
assembler = vecAssembler_4b256eb28ea0
dataDF = [season: int, yr: int ... 12 more fields]


[season: int, yr: int ... 12 more fields]

We need to convert the output column to label so that the algorithms can find it.

In [8]:
val dataDF1 = dataDF.withColumnRenamed("cnt", "label")

dataDF1 = [season: int, yr: int ... 12 more fields]


[season: int, yr: int ... 12 more fields]

In [9]:
dataDF1.show(3)

+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+
|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|label|            features|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+
|     1|  0|   1|  0|      0|      6|         0|         1|0.24|0.2879|0.81|      0.0| 16.0|[1.0,0.0,1.0,0.0,...|
|     1|  0|   1|  1|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0| 40.0|[1.0,0.0,1.0,1.0,...|
|     1|  0|   1|  2|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0| 32.0|[1.0,0.0,1.0,2.0,...|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+
only showing top 3 rows



In [10]:
val Array(train, test) = dataDF1.randomSplit(Array(0.8, 0.2))
println(train.count, rawData.count)

(13998,17380)


train = [season: int, yr: int ... 12 more fields]
test = [season: int, yr: int ... 12 more fields]


[season: int, yr: int ... 12 more fields]

In [11]:
train.show(3)

+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+
|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|label|            features|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+
|     1|  0|   1|  0|      0|      0|         0|         1|0.04|0.0758|0.57|   0.1045| 22.0|[1.0,0.0,1.0,0.0,...|
|     1|  0|   1|  0|      0|      0|         0|         1| 0.1|0.0758|0.42|   0.3881| 25.0|[1.0,0.0,1.0,0.0,...|
|     1|  0|   1|  0|      0|      0|         0|         1|0.16|0.1818| 0.8|   0.1045| 33.0|[1.0,0.0,1.0,0.0,...|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+
only showing top 3 rows



In [12]:
val lr = new LinearRegression()
    .setMaxIter(10)
    .setRegParam(0.3)
    .setElasticNetParam(0.8)

lr = linReg_842863b84be5


linReg_842863b84be5

In [13]:
//Fit the model
val lrModel = lr.fit(train)

lrModel = linReg_842863b84be5


linReg_842863b84be5

In [14]:
// Print the coefficients and intercept for linear regression
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9984920353821497] Intercept: 0.2855511662326547


In [15]:
// Summarize the model over the training set and print out some metrics
val trainingSummary = lrModel.summary
println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"objectiveHistory: [${trainingSummary.objectiveHistory.mkString(",")}]")
trainingSummary.residuals.show()
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"r2: ${trainingSummary.r2}")

numIterations: 11
objectiveHistory: [0.5,0.37538707896139756,0.06708558770721323,0.03654757299655967,0.0027323393627725354,0.0016662015968778327,0.0015255864698202242,0.0014903682755554226,0.0014865337463367911,0.001486491874784517,0.0014864749963246575]
+--------------------+
|           residuals|
+--------------------+
| -0.2523759446399474|
| -0.2478520507863955|
| -0.2357883338436011|
| -0.2267405461364973|
|-0.25991576772920055|
| -0.2749954139077033|
| -0.2780113431434037|
| -0.2780113431434037|
|-0.26745559081845016|
| -0.2780113431434037|
|-0.27197948467200206|
|-0.27650337852555307|
| -0.2749954139077033|
|-0.25991576772920055|
|-0.28102727237910363|
| -0.2749954139077033|
| -0.2689635554363008|
|-0.26594762620059953|
|-0.25991576772920055|
|-0.27197948467200206|
+--------------------+
only showing top 20 rows

RMSE: 0.27365377958826403
r2: 0.9999977260427113


trainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@48196be0


org.apache.spark.ml.regression.LinearRegressionTrainingSummary@48196be0

In [16]:
val predictions = lrModel.transform(test)
predictions.show(3)

+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+------------------+
|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|label|            features|        prediction|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+------------------+
|     1|  0|   1|  0|      0|      1|         1|         2|0.24|0.2273|0.65|   0.2239|  7.0|[1.0,0.0,1.0,0.0,...| 7.274995413907703|
|     1|  0|   1|  0|      0|      5|         1|         1|0.12|0.1364| 0.5|    0.194| 14.0|[1.0,0.0,1.0,0.0,...| 14.26443966158275|
|     1|  0|   1|  0|      0|      5|         1|         2|0.24|0.2273| 0.7|   0.2537| 21.0|[1.0,0.0,1.0,0.0,...|21.253883909257798|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+-----+--------------------+------------------+
only showing top 3 rows



predictions = [season: int, yr: int ... 13 more fields]


[season: int, yr: int ... 13 more fields]

In [17]:
val eval = new RegressionEvaluator().setMetricName("rmse").setLabelCol("label").setPredictionCol("prediction")
val rmse = eval.evaluate(predictions)

eval = regEval_ee8c23c83a47
rmse = 0.272956438102313


0.272956438102313